In [1]:
!pip install plotly
!pip install mplfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import mplfinance as mpf
import matplotlib.pyplot as plt
import requests
import plotly.graph_objects as go
from datetime import datetime
import numpy as np


Function to check for symbols

In [3]:
url = 'https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords=tata&apikey=SHEYWPL1ZYCDZ17D'
r = requests.get(url)
data = r.json()

print(data)

{'bestMatches': [{'1. symbol': 'TATACHEM.BSE', '2. name': 'TATA CHEMICALS LTD.', '3. type': 'Equity', '4. region': 'India/Bombay', '5. marketOpen': '09:15', '6. marketClose': '15:30', '7. timezone': 'UTC+5.5', '8. currency': 'INR', '9. matchScore': '0.5333'}, {'1. symbol': 'TATACOMM.BSE', '2. name': 'TATA COMMUNICATIONS LTD.', '3. type': 'Equity', '4. region': 'India/Bombay', '5. marketOpen': '09:15', '6. marketClose': '15:30', '7. timezone': 'UTC+5.5', '8. currency': 'INR', '9. matchScore': '0.5333'}, {'1. symbol': 'TATAELXSI.BSE', '2. name': 'Tata Elxsi Limited', '3. type': 'Equity', '4. region': 'India/Bombay', '5. marketOpen': '09:15', '6. marketClose': '15:30', '7. timezone': 'UTC+5.5', '8. currency': 'INR', '9. matchScore': '0.5000'}, {'1. symbol': 'TATAPOWER.BSE', '2. name': 'TATA POWER CO.LTD.', '3. type': 'Equity', '4. region': 'India/Bombay', '5. marketOpen': '09:15', '6. marketClose': '15:30', '7. timezone': 'UTC+5.5', '8. currency': 'INR', '9. matchScore': '0.5000'}, {'1. s

In [4]:
def fetch_daily_ohlc(symbol, apikey):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=compact&apikey={apikey}"
    r = requests.get(url)
    return r.json()


In [5]:
def calculate_percentage_differences(val1, val2):
    return abs(val1 - val2) / val2 * 100


def calculate_candle_length_percentage(open_price, close_price):
    return abs(close_price - open_price) / open_price * 100

In [139]:
def is_downtrend(candles, window=2):
    """Check for a downtrend using moving average of LOW prices"""
    low_prices =[float(c["3. low"]) for c in candles]

    if len(low_prices) < window:
        return True  # not enough data to form a trend

    moving_avg = [np.mean(low_prices[i:i+window]) for i in range(len(low_prices) - window + 1)]

    return all(earlier <= later for earlier, later in zip(moving_avg, moving_avg[1:]))

def is_uptrend(candles, window=2):
    """Check for an uptrend using moving average of LOW prices"""
    low_prices = [float(c["3. low"]) for c in candles]

    if len(low_prices) < window:
        return True  # not enough data to form a trend

    moving_avg = [np.mean(low_prices[i:i+window]) for i in range(len(low_prices) - window + 1)]

    return all(earlier >= later for earlier, later in zip(moving_avg, moving_avg[1:]))


In [67]:
def is_spinning(candle,
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0,
    body_percent=0.32,
    shadow_threshold=0.04):

    open_price = float(candle["1. open"])
    close_price = float(candle["4. close"])
    high_price = float(candle["2. high"])
    low_price = float(candle["3. low"])

    high_close_diff_pct = calculate_percentage_differences(high_price, close_price)
    open_low_diff_pct = calculate_percentage_differences(open_price, low_price)
    high_open_diff_pct = calculate_percentage_differences(high_price, open_price)
    close_low_diff_pct = calculate_percentage_differences(close_price, low_price)

    candle_length_pct = calculate_candle_length_percentage(open_price, close_price)

    total_range = abs(high_price-low_price)
    body_size = abs(open_price-close_price)
    upper_wick = high_price - max(open_price, close_price)
    lower_wick = min(open_price, close_price) - low_price

    upper_wick_pct = calculate_percentage_differences(high_price, max(open_price, close_price))
    lower_wick_pct = calculate_percentage_differences(min(open_price, close_price), low_price)
    # print(body_size,body_percent,total_range)
    if body_size > body_percent * total_range:
        return False

    if abs(upper_wick - lower_wick) > shadow_threshold*100:
        return False

    return True


def detect_spinningTop(data,
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0):
    time_series = data.get("Time Series (Daily)", {})
    results = []

    for date, candle in time_series.items():
        if is_spinning(
            candle,
            upper_wick_threshold,
            lower_wick_threshold,
            min_candle_length,
            max_candle_length
        ):
            results.append({
                "date": date,
                "open": float(candle["1. open"]),
                "high": float(candle["2. high"]),
                "low": float(candle["3. low"]),
                "close": float(candle["4. close"]),
                "volume": int(candle["5. volume"])
            })

    return results

In [33]:
def detect_all_spinning(data,
    upper_wick_threshold=1.0,
    lower_wick_threshold=1.0,
    min_candle_length=1.0,
    max_candle_length=10.0):
    print(data)
    results = detect_spinningTop(
        data,
        upper_wick_threshold,
        lower_wick_threshold,
        min_candle_length,
        max_candle_length
    )

    return pd.DataFrame(results)


In [34]:
def plot_candlestick_with_spinning(data, spinning_df):
    # Extract the time series data and convert it into a DataFrame
    time_series = data.get("Time Series (Daily)", {})

    # Convert the time series into a DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')

    # Convert all values to float and rename columns for mplfinance compatibility
    df = df[['1. open', '2. high', '3. low', '4. close', '5. volume']].astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    # Convert the index to datetime
    df.index = pd.to_datetime(df.index)
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

    fig.show()

In [ ]:
apikey = "SHEYWPL1ZYCDZ17D"
symbol = "SBIN.BSE"
data = fetch_daily_ohlc(symbol, apikey)

spinning_df = detect_all_spinning(
        data,
        upper_wick_threshold=0.5,
        lower_wick_threshold=0.5,
        min_candle_length=1.0,
        max_candle_length=10.0
)

print("Detected Spinning Candles:")
print(spinning_df)


In [123]:
plot_candlestick_with_spinning(data, spinning_df)

In [137]:

def calculate_profit_loss(
    data: dict,
    spinning_df: pd.DataFrame,
    lookback: int = 6,
    take_profit_pct=0.073,
    stop_pct: float = 0.06
):
    """
    For each spinning top date, enter next day:
      - If prior lookback days form downtrend, go LONG; else SHORT.
      - Exit when profit or loss reaches stop_pct of entry price.
      - Simulate day-by-day until exit.
    Returns total P/L and list of trades.
    """
    ts = data.get('Time Series (Daily)', {})
    df = pd.DataFrame.from_dict(ts, orient='index').astype(float)
    df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    df.index = pd.to_datetime(df.index)
    trades = []
    total_pnl=0

    dates = spinning_df['date']
    sorted_dates = [dt.strftime('%Y-%m-%d') for dt in df.index]
    # print(ts[sorted_dates[0]])
    # print(sorted_dates)
    for date in dates:
      idx = sorted_dates.index(date)
      prev_candle_str = sorted_dates[idx + 1 : idx + lookback]
      prev_candles = [ts[d] for d in prev_candle_str]

      if idx == 0:
            continue
      entry_idx   = idx - 1
      entry_dt    = df.index[entry_idx]
      entry_price = df.at[entry_dt, 'Open']
      close = df.at[entry_dt,'Close']
      # decide direction & thresholds
      if is_downtrend(prev_candles):
          direction       = 'long'
          profit_target   = entry_price * (1 + take_profit_pct)
          stop_loss_level = entry_price * (1 - stop_pct)
      else:
          direction       = 'short'
          profit_target   = entry_price * (1 - take_profit_pct)
          stop_loss_level = entry_price * (1 + stop_pct)

      print(direction,entry_dt)
      # now scan only **future** days: these are at indices entry_idx-1, entry_idx-2, …, 0
      exit_dt    = None
      exit_price = None
      for future_idx in range(entry_idx, -1, -1):
          trade_dt = df.index[future_idx]
          high = df.at[trade_dt, 'High']
          low  = df.at[trade_dt, 'Low']
          open = df.at[trade_dt,'Open']


          if direction == 'long' :
              if high >= profit_target:
                  exit_price = profit_target
                  exit_dt    = trade_dt
                  break
              if low <= stop_loss_level:
                  exit_price = stop_loss_level
                  exit_dt    = trade_dt
                  break
          else:  # short
              if low <= profit_target:
                  exit_price = profit_target
                  exit_dt    = trade_dt
                  break
              if high >= stop_loss_level:
                  exit_price = stop_loss_level
                  exit_dt    = trade_dt
                  break

      # fallback: exit at last available close (chronologically oldest)
      if exit_dt is None:
          exit_dt    = df.index[-1]
          exit_price = entry_price

      pnl = (exit_price - entry_price) if direction == 'long' else (entry_price - exit_price)
      total_pnl += pnl

      trades.append({
          'entry_date':  entry_dt,
          'entry_price': entry_price,
          'exit_date':   exit_dt,
          'exit_price':  exit_price,
          'direction':   direction,
          'pnl':         pnl
      })
    return total_pnl, trades

In [ ]:
total_profit_loss, trades = calculate_profit_loss(data, spinning_df)

print(f"Total Profit/Loss: {total_profit_loss}")
for trade in trades:
    print(f"Buy on {trade['entry_date']} at {trade['entry_price']}, Sell on {trade['exit_date']} at {trade['exit_price']}, Profit/Loss: {trade['pnl']}")